# **CNN으로 MNIST 분류하기**

## **1. MNIST란?**

- Modified National Institute of Standards and Technology 데이터셋의 약자
- 손글씨 숫자 이미지 (0~9) 인식을 위한 머신러닝/딥러닝 입문용 데이터셋
- 주로 이미지 분류(Classification) 실습에 사용

| 항목      | 내용                   |
| ------- | -------------------- |
| 이미지 크기  | 28×28 픽셀 (Grayscale) |
| 클래스 수   | 10개 (0\~9 숫자)        |
| 학습 데이터  | 60,000장              |
| 테스트 데이터 | 10,000장              |
| 채널      | 1 (흑백)               |


## **2. 모델 아키텍처**

- 1번 레이어 : 합성곱층(Convolutional layer)
```
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))
```
- 2번 레이어 : 합성곱층(Convolutional layer)
```
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))
```
- 3번 레이어 : 전결합층(Fully-Connected layer)
```
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

## **3. MNIST 분류 모델 구현하기**

### **기본 세팅 설정**

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init     # 가중치 초기화

In [ ]:
# CUDA 사용 : GPU를 이용해 연산을 가속할 수 있도록 설계된 기술
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### **학습 준비**

In [ ]:
# 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드

                         # ⭐⭐빈칸⭐⭐
                         transform=transforms._____(), # ⭐⭐텐서로 변환하기 위한 메소드를 작성해주세요
                         download=True)

# 데이터로더를 사용하여 배치 크기 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.41MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.8MB/s]


### **클래스로 모델 설계**

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # 첫번째층    배치 크기 × 채널 × 높이(height) × 너비(widht)
        # ImagIn shape =(100, 1, 28, 28)
        #    Conv     -> (100, 32, 28, 28)
        #    Pool     -> (100, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            # ⭐⭐빈칸 - 합성곱 층의 채널의 수⭐⭐
            torch.nn.Conv2d(1, ___, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImagIn shape =(100, 32, 14, 14)
        #    Conv      ->(100, 64, 14, 14)
        #    Pool      ->(100, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            # ⭐⭐빈칸 - 입력 이미지의 채널의 수⭐⭐
            torch.nn.Conv2d(___, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 정의 : 64x7x7 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

#### **⭐⭐문제⭐⭐**

- 첫번째 층에서 합성곱층과 풀링층의 커널 사이즈와 스트라이드 크기를 각각 작성해주세요.
  - 합성곱층
    - 커널 :
    - 스트라이드 :
  - 풀링층
    - 커널 :
    - 스트라이드 :

### **모델/비용함수/옵티마이저 정의**

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

# 비용함수 & 옵티마이저
criterion = torch.nn.CrossEntropyLoss().to(device)    # 출력값에 활성화함수(소프트맥스) 적용 + 비용 함수
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### **배치 수 확인**

In [ ]:
total_batch = len(data_loader)
print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


# 미니 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미

### **모델 훈련**

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0  # 에포크당 평균 비용을 저장하기 위한 변수 초기화

    for X, Y in data_loader:  # 미니 배치 단위로 데이터를 꺼내옴. X는 입력 데이터, Y는 레이블
        # 이미지 데이터는 이미 (28x28) 크기를 가지므로, 별도의 reshape 필요 없음
        # 레이블 Y는 원-핫 인코딩이 아닌 정수형 클래스 레이블임
        X = X.to(device)  # 입력 데이터를 연산이 수행될 장치로 이동 (예: GPU)
        Y = Y.to(device)  # 레이블을 연산이 수행될 장치로 이동 (예: GPU)

        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
        hypothesis = model(X)  # 모델을 통해 예측값(hypothesis)을 계산 (순전파 연산)
        cost = criterion(hypothesis, Y)  # 예측값과 실제값 Y 간의 손실(cost) 계산
        cost.backward()  # 역전파 연산을 통해 기울기 계산
        optimizer.step()  # 옵티마이저를 통해 파라미터 업데이트

        avg_cost += cost / total_batch  # 현재 배치의 비용을 전체 배치 수로 나누어 누적

    # 에포크가 끝날 때마다 평균 비용 출력
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225536346
[Epoch:    2] cost = 0.0630393252
[Epoch:    3] cost = 0.0463228337
[Epoch:    4] cost = 0.0375525318
[Epoch:    5] cost = 0.0313377567
[Epoch:    6] cost = 0.0260521118
[Epoch:    7] cost = 0.0216697622
[Epoch:    8] cost = 0.018259177
[Epoch:    9] cost = 0.0160413105
[Epoch:   10] cost = 0.0135012018
[Epoch:   11] cost = 0.0101749944
[Epoch:   12] cost = 0.0101597598
[Epoch:   13] cost = 0.0083907824
[Epoch:   14] cost = 0.00647852104
[Epoch:   15] cost = 0.00630554045


### **모델 예측**

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad() 사용
with torch.no_grad():
    # 테스트 데이터를 모델에 입력하기 위한 준비
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
    Y_test = mnist_test.test_labels.to(device)  # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

    # 모델 예측 수행
    prediction = model(X_test)  # 테스트 데이터에 대해 모델이 예측한 결과값

    # 예측 결과와 실제 레이블 비교
    correct_prediction = torch.argmax(prediction, 1) == Y_test  # 예측된 클래스와 실제 레이블이 일치하는지 확인

    # 정확도 계산
    accuracy = correct_prediction.float().mean()  # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함
    print('Accuracy:', accuracy.item())  # 정확도를 출력

Accuracy: 0.988599956035614


/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


#### **⭐⭐문제⭐⭐**

- 합성곱층이 늘어나면 정확도는 어떻게 될지 간단하게 작성해보세요!
  - 정답 :